In [1]:
import pandas as pd
import re

In [2]:
bank_statement_df = pd.read_excel(r"D:\NPS_phase1\nps-reconciliation\output\BANK STMT PHASE 4\Bank_Manjushree.xlsx",skiprows=1)

In [3]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,9200008062,-2403.56,C,PRABHUMONEY#PMT9173,NPS-IF-3133982,NIBLNPKTOff,2023-08-29T00:00:00,NaN
1,2,9200008062,7000.00,D,9842062777,"100003988398,NPS",42120,2023-08-29T00:00:00,NaN
2,3,9200008062,1400.00,D,9811026261,"100003989820,NPS",42493,2023-08-29T00:00:00,NaN
3,4,9200008062,-41295.83,C,PRABHUMONEY#PMT9193,NPS-IF-3137847,NIBLNPKT,2023-08-29T00:00:00,NaN
4,5,9200008062,-10000.00,C,PRABHUMONEY#PMT1133,NPS-IF-3137771,NIBLNPKT,2023-08-29T00:00:00,NaN


In [4]:
for index, row in bank_statement_df.iterrows():
    # print(bank_statement_df.iloc[index]['Transaction ID'])
    for roww in (str(row['Desc1']) ,str( row['Desc2'])):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            break
        
        
        elif "10000" in roww:
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]

In [5]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id
0,1,9200008062,-2403.56,C,PRABHUMONEY#PMT9173,NPS-IF-3133982,NIBLNPKTOff,2023-08-29T00:00:00,NaN,3133982
1,2,9200008062,7000.00,D,9842062777,"100003988398,NPS",42120,2023-08-29T00:00:00,NaN,3988398
2,3,9200008062,1400.00,D,9811026261,"100003989820,NPS",42493,2023-08-29T00:00:00,NaN,3989820
3,4,9200008062,-41295.83,C,PRABHUMONEY#PMT9193,NPS-IF-3137847,NIBLNPKT,2023-08-29T00:00:00,NaN,3137847
4,5,9200008062,-10000.00,C,PRABHUMONEY#PMT1133,NPS-IF-3137771,NIBLNPKT,2023-08-29T00:00:00,NaN,3137771
5,6,9200008062,-75000.00,C,PRABHUMONEY#PMT9206,NPS-IF-3142810,NIBLNPKT,2023-08-29T00:00:00,NaN,3142810


In [9]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'D' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'C':
            mode = 'CR'
            amount = abs(row['Amount'])
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [10]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [11]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,9200008062,2403.56,C,PRABHUMONEY#PMT9173,NPS-IF-3133982,NIBLNPKTOff,2023-08-29T00:00:00,NaN,3133982,CR
1,2,9200008062,7000.00,D,9842062777,"100003988398,NPS",42120,2023-08-29T00:00:00,NaN,3988398,DR
2,3,9200008062,1400.00,D,9811026261,"100003989820,NPS",42493,2023-08-29T00:00:00,NaN,3989820,DR
3,4,9200008062,41295.83,C,PRABHUMONEY#PMT9193,NPS-IF-3137847,NIBLNPKT,2023-08-29T00:00:00,NaN,3137847,CR
4,5,9200008062,10000.00,C,PRABHUMONEY#PMT1133,NPS-IF-3137771,NIBLNPKT,2023-08-29T00:00:00,NaN,3137771,CR
5,6,9200008062,75000.00,C,PRABHUMONEY#PMT9206,NPS-IF-3142810,NIBLNPKT,2023-08-29T00:00:00,NaN,3142810,CR


In [12]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [13]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,9200008062,2403.56,C,PRABHUMONEY#PMT9173,NPS-IF-3133982,NIBLNPKTOff,2023-08-29,NaN,3133982,CR
1,2,9200008062,7000.00,D,9842062777,"100003988398,NPS",42120,2023-08-29,NaN,3988398,DR
2,3,9200008062,1400.00,D,9811026261,"100003989820,NPS",42493,2023-08-29,NaN,3989820,DR
3,4,9200008062,41295.83,C,PRABHUMONEY#PMT9193,NPS-IF-3137847,NIBLNPKT,2023-08-29,NaN,3137847,CR
4,5,9200008062,10000.00,C,PRABHUMONEY#PMT1133,NPS-IF-3137771,NIBLNPKT,2023-08-29,NaN,3137771,CR
